### Operación remota de generadores de funciones de la marca **Siglent**

De la marca Siglent, en el laboratorio 3 están disponible los modelos:
- SDG1050
- SDG1010

La idea de este tutorial es mostrar como se operan estos instrumentos a partir del standart NI-VISA utilizando como interfaz la [librería pyvisa](https://pyvisa.readthedocs.io/en/latest/) de Python. Ambos modelos utilizan el mismo lenguaje.

[Ver manual de programación de SDG serie 1000](https://asignaturas.df.uba.ar/l3-raspa/wp-content/uploads/sites/26/2025/02/SDG-Programming-Guide-.pdf).

Empezamos importando las librerías necesarias:

In [ ]:
import pyvisa                               # para comunicarnos con los equipos
import numpy as np                          # manejo de arrays
import time                                 # para hacer pausas entre mediciones

En primer lugar, necesitamos un objeto que nos permita comunicarnos e interactuar con los instrumentos que tenemos conectados vía puerto USB, GPIB, Serial (RS232), Ethernet, etc. Ese objeto es el *resource manager* (administrador de recursos) y la invocamos desde la librería pyvisa:

In [ ]:
rm = pyvisa.ResourceManager()

El resource manager (rm) se utiliza para: (a) detectar dispositivos conectados y listar todos los recursos disponibles y (b) crear interfases para comunicarnos con dispositivos.

Una vez definido el objeto **rm**, podemos listar los instrumentos conectados a nuestra PC: 

In [ ]:
instrumentos_conectados = rm.list_resources()           # lista de instrumentos conectados

print (instrumentos_conectados)

De la lista obtenida en la celda anterior, buscamos el generador. Copiamos el nombre y lo pegamos en la variable instrumento, luego procedemos a establecer la comunicación con el equipo:

In [ ]:
instrumento = 'puerto::marca::modelo::NroSerie'         # colocar acá el instrumento asociado al osciloscopio 

gen = rm.open_resource(instrumento)                     # Establezco comunicación con el osciloscopio

gen.query('*IDN?')                                      # le pedimos que se identifique para checkear conexión

El objeto *gen* será nuestra interfase, éste nos permite interactuar con el generador de funciones utilizando rutinas estándar. Generalmente, realizamos dos tipos de operaciones:

- Enviar un comando al instrumento para configurar parámetros o iniciar procesos sin esperar ninguna respuesta. Esto se realiza mediante rutinas de tipo **write**.
  
- Enviar un comando al instrumento y esperar una respuesta, como por ejemplo, obtener los datos que estoy midiendo o consultar el estado del instrumento. Esto se realiza mediante rutinas de tipo **query**.

El lenguaje que utilizamos para comunicarnos con el equipo está definido en el estándar [**SCPI**](https://en.wikipedia.org/wiki/Standard_Commands_for_Programmable_Instruments) (Standard Commands for Programmable Instruments), el cual es ampliamente utilizado para la comunicación remota con equipos de instrumentación como osciloscopios, multímetros y generadores de funciones.

### 1- Generación de funciones

Establecida la conexión con el generador de función, a continuación mostramos como configurar el equipo para que entregue distintos tipos de señales.

#### 1.1. Señal sinusoidal 

En primer lugar, apago la salida. Esto es opcional pero es una buena costumbre:

In [ ]:
gen.write('C1:OUTP OFF')  

Para generar una función sinusoide necesitamos definir tres parámetros: Amplitud, frecuencia y fase, esto dependerá del experimento. Por ejemplo, supongamos:

In [ ]:
ampli= 4.            # voltaje pico a pico
freq = 1000.         # en Hz 
fase = 180.          # en grados

configuramos la señal:

In [ ]:
gen.write('C1:BSWV WVTP,SINE,FRQ,%.2f,AMP,%.2f,PHSE,%.2f'% (freq, ampli, fase)   ) 

**Descomposición del comando:**

- C1: Indica que la configuración es para el canal 1 del generador (usar C2: para el canal 2).

- BSWV (Basic Waveform): Especifica que estamos configurando los parámetros básicos de una señal de forma de onda.

- WVTP,SINE (Wave Type): Define el tipo de señal. En este caso, una onda sinusoidal (SINE). 

    - Otros posibles valores son:

        - SQUARE (Cuadrada)
        - RAMP (Diente de sierra o triangular)
        - PULSE (Pulso)
        - NOISE (Ruido)
        - DC (Nivel constante)
        - ARB (Forma de onda arbitraria)
        - FRQ,1000 (Frequency): Define la frecuencia de la onda en Hz. En este caso, es 1000 Hz (1 kHz). Se puede cambiar por cualquier valor dentro del rango permitido por el generador.

- AMP,1 (Amplitude):Establece la amplitud pico a pico (Vpp) de la señal. En este caso, es 1 Vpp.También se puede especificar como RMS usando AMPVRMS


volvemos a activar el canal para que empiece a generar:

In [ ]:
gen.write('C1:OUTP ON') 

#### 1.2. Señal tipo Rampa

Queremos una señal tipo rampa o diente de sierra. Este tipo de señal crece linealmente por un tiempo para luego decaer linealmente. Los parámetros frecuencia y amplitud son análogos a la señal tipo seno, el parámetro distinto es la simetría que indica el porcentaje de tiempo que la señal esta subiendo. Con simetría 50% la señal tiene el mismo tiempo de bajada que de subida, con simetría 30% le damos solo 30% del tiempo a la subida.

In [ ]:
# Detener la señal antes de configurar
gen.write('C1:OUTP OFF')

# Configurar la señal tipo rampa
gen.write('C1:BSWV WVTP,RAMP,FRQ,1000,AMP,1,SYM,30')  # 1 kHz, 1 Vpp, Simetría 30%

# Encender la señal
gen.write('C1:OUTP ON')

### 2. Barrido en frecuencias

Vamos a hacer un barrido en frecuencia de una función sinusoidal, la seteamos en el generador:

In [ ]:
gen.write('C1:BSWV WVTP,SINE' ) 

En primer lugar definimos las frecuencias mínimas y máximas que queremos, así como el numero de freq a explorar en el barrido.

In [ ]:
frec1 = 10.                # freq mínima
frec2 = 1000.              # freq máxima

Primero, exploramos un barrido en frecuencias lineal. En este caso, variamos la frecuencia tal que esta se incrementa de manera uniforme entre un valor inicial y un valor final. 

In [ ]:
paso= 100                                          # espaciamiento entre freqs

N1 = int( (frec2-frec1)/paso ) +1                  # cant de freqs en el conjunto

frecuencias = np.linspace( frec1, frec2, N1 )      # conjunto de freqs


for freq in frecuencias:
    print (freq)
    gen.write('C1:BSWV FRQ,%f' % freq)            # fija la freq en el gen
    time.sleep(2)

Ahora planteamos un barrido logarítmico. En este caso, las frecuencias de prueba se distribuyen de manera que la relación entre frecuencias consecutivas es constante, en lugar de la diferencia absoluta. Esto permite cubrir un rango amplio de frecuencias con mayor detalle en bajas frecuencias y menos puntos en altas frecuencias, siendo útil en análisis de respuesta en frecuencia y caracterización de sistemas que siguen comportamientos exponenciales o de amplio espectro.

In [ ]:
frec1 = 10.                # freq mínima
frec2 = 1000.              # freq máxima

N2=100

frecuencias=np.geomspace( frec1, frec2, N2 )

for freq in frecuencias:
    print (freq)
    gen.write('C1:BSWV FRQ,%f' % freq)             # fija freq
    time.sleep(2)                                  # da tiempo al instrumento para trabajar